In [1]:
import pandas as pd

In [4]:
data = pd.read_parquet('../data/hugging/train-00000-of-00006-1d1ab78e9df29da3.parquet',engine='pyarrow')

In [5]:
data.head()

,text,labels,encoded_labels
0,Football positions \n\nHi Mehudson. When you'...,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,Thank you SO MUCH for your Nazi-like oversight...,"[toxic, profane, insult]","[1, 1, 1, 0, 0, 0, 0, 0, 0]"
2,Darkwind \n Stick it up your arse you offensi...,"[toxic, profane]","[1, 1, 0, 0, 0, 0, 0, 0, 0]"
3,Message from you to an IP \n\nI received a mes...,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,Misleading and inappropriate citations Used,[],"[0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057492 entries, 0 to 1057491
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   text            1057492 non-null  object
 1   labels          1057492 non-null  object
 2   encoded_labels  1057492 non-null  object
dtypes: object(3)
memory usage: 24.2+ MB


In [7]:
data.isnull().sum()

text              0
labels            0
encoded_labels    0
dtype: int64